In [1]:
# add mappings
import requests
mappings  = {
    "mappings": {
        "_doc" : {
            "properties" : {
                "suggest" : {
                    "type" : "completion"
                },
                "title" : {
                    "type": "keyword"
                }
            }
        }
    }
}
requests.put('http://localhost:9200/names', json=mappings)

<Response [400]>

In [2]:
# INDEXING similar terms

terms = {
  "suggest" : ['jonas', 'jonatas', 'jonathan', 'jones']
}
res = requests.put('http://localhost:9200/names/_doc/similar?refresh', json=terms)
print(res.text)

{"_index":"names","_type":"_doc","_id":"similar","_version":4,"result":"updated","forced_refresh":true,"_shards":{"total":2,"successful":1,"failed":0},"_seq_no":3,"_primary_term":1}


In [3]:
# querying similar terms FUZZY ON -- typo FIXED

query = {
    "_source": True,
    "suggest": {
        "name-suggest" : {
            "prefix" : "jonax", 
            "completion" : { 
                "field" : "suggest",                
                "fuzzy" : {
                    "fuzziness" : 3
                },
                "size": 5
            }
        }
    }
}
res = requests.post('http://localhost:9200/names/_search?pretty', json=query)
print(res.text)

{
  "took" : 6,
  "timed_out" : false,
  "_shards" : {
    "total" : 5,
    "successful" : 5,
    "skipped" : 0,
    "failed" : 0
  },
  "hits" : {
    "total" : 0,
    "max_score" : 0.0,
    "hits" : [ ]
  },
  "suggest" : {
    "name-suggest" : [
      {
        "text" : "jonax",
        "offset" : 0,
        "length" : 5,
        "options" : [
          {
            "text" : "jonas",
            "_index" : "names",
            "_type" : "_doc",
            "_id" : "similar",
            "_score" : 3.0,
            "_source" : {
              "suggest" : [
                "jonas",
                "jonatas",
                "jonathan",
                "jones"
              ]
            }
          }
        ]
      }
    ]
  }
}



In [8]:
# querying similar terms FUZZY ON -- 
# NO TYPO, BUT FUZZY STILL RETURN 'jonas' when 'jonatas' would be more appropriated

query = {
    "_source": True,
    "suggest": {
        "name-suggest" : {
            "prefix" : "jonat", 
            "completion" : { 
                "field" : "suggest",                
                "fuzzy" : {
                    "fuzziness" : 3
                },
                "size": 5
            }
        }
    }
}
res = requests.post('http://localhost:9200/names/_search?pretty', json=query)
print(res.text)

{
  "took" : 5,
  "timed_out" : false,
  "_shards" : {
    "total" : 5,
    "successful" : 5,
    "skipped" : 0,
    "failed" : 0
  },
  "hits" : {
    "total" : 0,
    "max_score" : 0.0,
    "hits" : [ ]
  },
  "suggest" : {
    "name-suggest" : [
      {
        "text" : "jonat",
        "offset" : 0,
        "length" : 5,
        "options" : [
          {
            "text" : "jonas",
            "_index" : "names",
            "_type" : "_doc",
            "_id" : "similar",
            "_score" : 3.0,
            "_source" : {
              "suggest" : [
                "jonas",
                "jonatas",
                "jonathan",
                "jones"
              ]
            }
          }
        ]
      }
    ]
  }
}

